In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote

from PIL import Image
from io import BytesIO
import re

commons_site = pb.Site("commons", "commons")

In [3]:
from modules.wmtools import is_commons_file, get_hash, upload_to_commons

In [4]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [12]:
url = 'http://premsa.gencat.cat/pres_fsvp/AppJava/governacio-relacions-institucionals/notapremsavw/301618/ca/la-consellera-de-governacio-el-sindic-daran-signen-lacord-financament-2017-del-conselh-generau.do'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
headers = {'User-Agent' : user_agent}
excluded = []
categories = ['June 2017 in Catalonia',
             'Meritxell Borràs',
             'Carles Barrera Sánchez']
cat_string = '\n'.join(['[[Category:'+i+']]' for i in categories])

r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [13]:
pub_date=soup.find_all("span", attrs={"itemprop": "datePublished"})[0].get_text().strip().split(' ')[0].split('-')
pub_date.reverse()
pub_date='-'.join(pub_date)
pub_date

'2017-06-17'

In [14]:
title = soup.find_all("h1", class_="FW_headline")[0].get_text().strip().replace('  ', ' ')
#title = u""
title

"La consellera de Governació i el Síndic d'Aran signen l'acord de finançament de 2017 del Conselh Generau d'Aran"

In [15]:
article_content = soup.find_all("div", class_="FW_article-content")[0].get_text().strip().split('\n')[0]
#article_content = u"El conseller d’Agricultura, Alimentació i Acció Rural, Joaquim Llena, acompanyat de la directora general d’Agricultura i Ramaderia, Rosa Cubel, i del director dels Serveis Territorials del DAR a Barcelona, Jaume Balagué, ha presidit, avui, l’acte de cloenda de l’Assemblea General Ordinària de la Federació Catalana d'Indústries de la Carn (FECIC), i el dinar posterior."
article_content

'La consellera de Governació, Administracions Públiques i Habitatge,\xa0Meritxell Borràs, i el Síndic del Conselh Generau d’Aran,\xa0Carlos Barrera, han signat aquest matí l’acord que recull el finançament corresponent a l’exercici 2017 del Conselh Generau d’Aran. Les transferències de la Generalitat, per un import total de 23.474.428 euros, estan destinades al finançament de la institució d’autogovern aranesa i a l’exercici de les seves competències pròpies, en compliment de la Llei 1/2015, del\xa05 de febrer, del règim especial d’Aran. Aquesta xifra representa un increment d’1.407.701 euros respecte del 2016.\r'

In [16]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{ca|1=${description}}}
|date=${date}
|source=[${url} Nota de Premsa - ${title}]
|author=Generalitat de Catalunya
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-gencat}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

vars = {
    "url": url,
    "description": article_content,
    "date": pub_date,
    "title": title,
    "cat_string": cat_string
}
t = Template(template)
_text = t.render(**vars)
_text

"=={{int:filedesc}}==\n{{Information\n|description={{ca|1=La consellera de Governació, Administracions Públiques i Habitatge,\xa0Meritxell Borràs, i el Síndic del Conselh Generau d’Aran,\xa0Carlos Barrera, han signat aquest matí l’acord que recull el finançament corresponent a l’exercici 2017 del Conselh Generau d’Aran. Les transferències de la Generalitat, per un import total de 23.474.428 euros, estan destinades al finançament de la institució d’autogovern aranesa i a l’exercici de les seves competències pròpies, en compliment de la Llei 1/2015, del\xa05 de febrer, del règim especial d’Aran. Aquesta xifra representa un increment d’1.407.701 euros respecte del 2016.\r}}\n|date=2017-06-17\n|source=[http://premsa.gencat.cat/pres_fsvp/AppJava/governacio-relacions-institucionals/notapremsavw/301618/ca/la-consellera-de-governacio-el-sindic-daran-signen-lacord-financament-2017-del-conselh-generau.do Nota de Premsa - La consellera de Governació i el Síndic d'Aran signen l'acord de finançamen

In [17]:
images = [{"url": image["href"].strip(), "name": image["title"].replace(':', ' -').replace('  ', ' ').strip()} for image in soup.find_all("a", class_="external") if '.jpg' in image['href'].lower()]
images

[{'name': 'Foto 1 (Autor - Gorka Martínez Llurda)',
  'url': 'http://www.gencat.cat/big/img/680/BIG_680564812061917_03.jpg'},
 {'name': 'Foto 2 (Autor - Gorka Martínez Llurda)',
  'url': 'http://www.gencat.cat/big/img/755/BIG_755173410061917_03.jpg'},
 {'name': 'Foto 3 (Autor - Gorka Martínez Llurda)',
  'url': 'http://www.gencat.cat/big/img/125/BIG_125572112061917_03.jpg'},
 {'name': 'Foto 4 (Autor - Departament Governació)',
  'url': 'http://www.gencat.cat/big/img/181/BIG_181075818061917_03.jpg'},
 {'name': 'Foto 5 (Autor - Departament de Governació)',
  'url': 'http://www.gencat.cat/big/img/165/BIG_165100102062017_03.jpg'}]

In [18]:
images = [{"name": "La consellera de Governació i el Síndic d'Aran signen l'acord de finançament de 2017 del Conselh Generau d'Aran 1 (Autor - Gorka Martínez Llurda)",
  "url": "http://www.gencat.cat/big/img/680/BIG_680564812061917_03.jpg"},
 {"name": "La consellera de Governació i el Síndic d'Aran signen l'acord de finançament de 2017 del Conselh Generau d'Aran 2 (Autor - Gorka Martínez Llurda)",
  "url": "http://www.gencat.cat/big/img/755/BIG_755173410061917_03.jpg"},
 {"name": "La consellera de Governació i el Síndic d'Aran signen l'acord de finançament de 2017 del Conselh Generau d'Aran 3 (Autor - Gorka Martínez Llurda)",
  "url": "http://www.gencat.cat/big/img/125/BIG_125572112061917_03.jpg"},
 {"name": "La consellera de Governació i el Síndic d'Aran signen l'acord de finançament de 2017 del Conselh Generau d'Aran 4 (Autor - Departament Governació)",
  "url": "http://www.gencat.cat/big/img/181/BIG_181075818061917_03.jpg"},
 {"name": "La consellera de Governació i el Síndic d'Aran signen l'acord de finançament de 2017 del Conselh Generau d'Aran 5 (Autor - Departament de Governació)",
  "url": "http://www.gencat.cat/big/img/165/BIG_165100102062017_03.jpg"}]

In [19]:
used_names = []

for i, image in enumerate(images):
    if i in excluded:
        continue
        
    # image download
    image_url = quote(image["url"].encode('utf-8'), ':/')
    image_name = image["name"].replace(':', ' -').replace('  ', ' ') + '.jpg'
    try: 
        image_path = os.path.join(images_directory, image_name)
        r = requests.get(image_url, headers=headers)
        image = Image.open(BytesIO(r.content))
        image.save(image_path)
    except :
        print ('Failed download. Skipping')
        continue

    # image already in Commons
    if is_commons_file(get_hash(image_path)) :
        print ("Already in commons. Skipping")
        os.remove(image_path)
        continue

    # management of image name at Commons
    # the image name is already used in commons
    if pb.Page(commons_site, "File:"+image_name).exists():
        print ("Image name already used in Commons")
        used_names.append(image_name)

    if image_name in used_names :
        counter = 1
        while True:
            regexp = re.compile(r'[0-9]{2}\.jpg')
            if regexp.search(image_name):
                image_name = re.sub(r'[0-9]{2}\.jpg', 
                                    '%02d.jpg' %counter, 
                                    image_name)
            else:
                # second image in the name series
                image_name = image_name.replace('.jpg', ' 01.jpg')
            counter += 1
            if image_name not in used_names and not pb.Page(commons_site, "File:"+image_name).exists():
                break
            else :
                used_names.append(image_name)
    else :
        used_names.append(image_name)

    # image upload
    bot = UploadRobot(image_path,
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()
    
    os.remove(image_path)         

The suggested description is:
=={{int:filedesc}}==
{{Information
}}escription={{ca|1=La consellera de Governació, Administracions Públiques i Habitatge, Meritxell Borràs, i el Síndic del Conselh Generau d’Aran, Carlos Barrera, han signat aquest matí l’acord que recull el finançament corresponent a l’exercici 2017 del Conselh Generau d’Aran. Les transferències de la Generalitat, per un import total de 23.474.428 euros, estan destinades al finançament de la institució d’autogovern aranesa i a l’exercici de les seves competències pròpies, en compliment de la Llei 1/2015, del 5 de febrer, del règim especial d’Aran. Aquesta xifra representa un increment d’1.407.701 euros respecte del 2016.
|date=2017-06-17
|source=[http://premsa.gencat.cat/pres_fsvp/AppJava/governacio-relacions-institucionals/notapremsavw/301618/ca/la-consellera-de-governacio-el-sindic-daran-signen-lacord-financament-2017-del-conselh-generau.do Nota de Premsa - La consellera de Governació i el Síndic d'Aran signen l'acord d

Upload successful.
Upload of La consellera de Governació i el Síndic d'Aran signen l'acord de finançament de 2017 del Conselh Generau d'Aran 5 (Autor - Departament de Governació).jpg successful.

1 pages read
1 pages written
Script terminated successfully.
